# Evaluate how well the Harmondy batch correction worked using 

In [2]:
!date

Mon Nov 10 21:22:29 UTC 2025


#### import libraries and set notebook variables

In [4]:
import scanpy as sc
from sklearn.metrics import silhouette_score
from anndata import AnnData
from scib_metrics.benchmark import Benchmarker, BioConservation, BatchCorrection
import warnings
warnings.simplefilter('ignore')

In [3]:
# variables and constants
cohort = 'aging'
assay = 'RNA'
DEBUG = True

# directories
wrk_dir = '/home/jupyter/brain_aging_phase1'
quants_dir = f'{wrk_dir}/demux'

# in files
in_h5ad = f'{quants_dir}/{cohort}.pegasus.leiden_085.subclustered.h5ad'

if DEBUG:
    print(in_h5ad)

/home/jupyter/brain_aging_phase1/demux/aging.pegasus.leiden_085.subclustered.h5ad


#### functions

In [5]:
def peek_anndata(adata: AnnData, message: str=None, verbose: bool=False):
    if not message is None and len(message) > 0:
        print(message)
    print(adata)
    if verbose:
        display(adata.obs.head())
        display(adata.var.head())

## read the anndata (h5ad) file

In [6]:
%%time
adata = sc.read_h5ad(in_h5ad)
peek_anndata(adata, 'loaded input anndata', DEBUG)

loaded input anndata
AnnData object with n_obs × n_vars = 167945 × 35441
    obs: 'pool_name', 'Sample_id', 'Tissue_source', 'Brain_region', 'Clinical_diagnosis', 'Age', 'Sex', 'donor_id', 'lane_num', 'Channel', 'n_genes', 'n_counts', 'percent_mito', 'scale', 'Group', 'leiden_labels', 'anno', 'leiden_labels_085', 'new_anno', 'Age_group', 'broad_celltype'
    var: 'n_cells', 'percent_cells', 'robust', 'highly_variable_features', 'mean', 'var', 'hvf_loess', 'hvf_rank'
    uns: 'Channels', 'Groups', 'PCs', 'W_diffmap', 'W_pca_harmony', 'broad_celltype_colors', 'c2gid', 'df_qcplot', 'diffmap_evals', 'diffmap_knn_distances', 'diffmap_knn_indices', 'genome', 'gncells', 'leiden_resolution', 'modality', 'ncells', 'new_anno_colors', 'norm_count', 'pca', 'pca_features', 'pca_harmony_knn_distances', 'pca_harmony_knn_indices', 'stdzn_max_value', 'stdzn_mean', 'stdzn_std'
    obsm: 'X_diffmap', 'X_fle', 'X_pca', 'X_pca_harmony', 'X_phi', 'X_umap', 'X_umap_085'
    varm: 'de_res', 'gmeans', 'gstds',

,pool_name,Sample_id,Tissue_source,Brain_region,Clinical_diagnosis,Age,Sex,donor_id,lane_num,Channel,...,n_counts,percent_mito,scale,Group,leiden_labels,anno,leiden_labels_085,new_anno,Age_group,broad_celltype
barcodekey,,,,,,,,,,,,,,,,,,,,,
AAACCCACAGTTGGTT-1,P001,Aging080,NIMH Human Brain Collection Core,Middle temporal gyrus,Healthy,21.5,Female,NHBCC-1187,1,Aging080,...,40076.0,0.197125,2.495321,one_group,6,Glutamatergic neuron LAMP5,6,ExN CUX2 LAMP5,young,ExN
AAACCCATCTCGCTTG-1,P001,Aging073,NIMH Human Brain Collection Core,Middle temporal gyrus,Healthy,64.3,Female,NHBCC-1119,1,Aging073,...,16716.0,0.424743,5.983008,one_group,6,Glutamatergic neuron LAMP5,6,ExN CUX2 LAMP5,old,ExN
AAACGAACACGGTAGA-1,P001,Aging095,NIMH Human Brain Collection Core,Middle temporal gyrus,Healthy,27.2,Male,NHBCC-1604,1,Aging095,...,4130.0,0.217918,24.213075,one_group,1,Oligodendrocyte,1,Oligodendrocyte-1,young,Oligodendrocyte
AAACGCTAGAACTCCT-1,P001,Aging073,NIMH Human Brain Collection Core,Middle temporal gyrus,Healthy,64.3,Female,NHBCC-1119,1,Aging073,...,36964.0,0.097392,2.705481,one_group,6,Glutamatergic neuron LAMP5,6,ExN CUX2 LAMP5,old,ExN
AAACGCTAGTCAGCCC-1,P001,Aging073,NIMH Human Brain Collection Core,Middle temporal gyrus,Healthy,64.3,Female,NHBCC-1119,1,Aging073,...,33798.0,0.278123,2.959018,one_group,6,Glutamatergic neuron LAMP5,6,ExN CUX2 LAMP5,old,ExN


,n_cells,percent_cells,robust,highly_variable_features,mean,var,hvf_loess,hvf_rank
featurekey,,,,,,,,
MIR1302-2HG,14,0.008336,False,False,0.000150,0.000293,0.000000,-1
OR4F5,10,0.005954,False,False,0.000100,0.000187,0.000000,-1
AL627309.1,2520,1.500491,True,False,0.028111,0.059149,0.065533,43514
AL627309.3,15,0.008931,False,False,0.000173,0.000399,0.000000,-1
AL627309.2,1,0.000595,False,False,0.000007,0.000008,0.000000,-1


CPU times: user 1.74 s, sys: 15.3 s, total: 17 s
Wall time: 17 s


### take a look at the cell counts by cell type

In [8]:
# get cell counts by 'categories'
categories = ['Brain_region', 'new_anno', 'leiden_labels_085']
for this_cat in categories:
    print(adata.obs[this_cat].value_counts())

Entorhinal cortex        54399
Putamen                  49625
Subventricular zone      36610
Middle temporal gyrus    27311
Name: Brain_region, dtype: int64
Oligodendrocyte-1    36479
SPN D1               15379
SPN D2               15280
Oligodendrocyte-2    12844
Astrocyte            11289
ExN CUX2 LAMP5        9055
uncertain             7548
InN ADARB2 VIP        7014
ExN FEZF2             6842
OPC                   6682
ExN RORB THEMIS       5130
InN LHX6 PVALB        4566
Microglia             3265
InN ADARB2 LAMP5      3216
ExN CUX2 ADARB2       3066
InN LHX6 SST          2937
uncertain-2           2677
SPN D1-2              2388
ExN RORB              1966
ExN LAMP5             1963
uncertain-3           1929
Ependymal             1853
Mural                 1293
Endothelial            953
Oligodendrocyte-3      806
Astrocyte-GFAP-Hi      599
SPN D2-2               465
ExN THEMIS             461
Radial Glia              0
Name: new_anno, dtype: int64
1     36479
2     15379
3     1

## Calculate silhouette score

In [16]:
%%time
sil_score = silhouette_score(adata.obsm['X_pca_harmony'], adata.obs['new_anno'])
print("Silhouette score after:", sil_score)
sil_score = silhouette_score(adata.obsm['X_pca'], adata.obs['new_anno'])
print("Silhouette score before:", sil_score)

Silhouette score after: 0.19709545
Silhouette score before: 0.19330893
CPU times: user 2h 7min 2s, sys: 2h 15min 10s, total: 4h 22min 13s
Wall time: 12min 2s


In [17]:
%%time
sil_score = silhouette_score(adata.obsm['X_pca_harmony'], adata.obs['pool_name'])
print("Silhouette score after:", sil_score)
sil_score = silhouette_score(adata.obsm['X_pca'], adata.obs['pool_name'])
print("Silhouette score before:", sil_score)

Silhouette score after: -0.08552128
Silhouette score before: 0.009827746
CPU times: user 2h 10min 31s, sys: 2h 15min 8s, total: 4h 25min 39s
Wall time: 14min 11s


In [12]:
%%time
sil_score = silhouette_score(adata.obsm['X_pca_harmony'], adata.obs['Sample_id'])
print("Silhouette score:", sil_score)

Silhouette score: -0.19136772
CPU times: user 1h 6min 51s, sys: 1h 10min 37s, total: 2h 17min 28s
Wall time: 6min 12s


In [15]:
%%time
sil_score = silhouette_score(adata.obsm['X_pca_harmony'], adata.obs['Brain_region'])
print("Silhouette score:", sil_score)

Silhouette score: -0.017677471
CPU times: user 1h 5min 2s, sys: 1h 7min 59s, total: 2h 13min 1s
Wall time: 6min 55s


## Use Benchmarker to calculate multiple metrics including LISI

In [ ]:
bm = Benchmarker(
    adata,
    batch_key='pool_name',
    label_key='new_anno',
    bio_conservation_metrics=BioConservation(),
    batch_correction_metrics=BatchCorrection(),
    embedding_obsm_keys=['X_pca', 'X_pca_harmony'],
    n_jobs=24,
)

bm.prepare()
bm.benchmark()

### get the benchmark results

In [ ]:
results = bm.get_results(min_max_scale=False)
print(results)

In [ ]:
bm.plot_results_table()

In [ ]:
bm.plot_results_table(min_max_scale=False)

In [ ]:
!date